In [0]:
from google.colab import drive
from os import chdir, getcwd

drive.mount('/content/gdrive')
chdir("/content/gdrive/My Drive/SSD_ADAS/ssd_keras/")
print("Working directory:", getcwd())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Working directory: /content/gdrive/My Drive/SSD_ADAS/ssd_keras


In [0]:
#Classes to include:
classes = [1] #person

from data_generator.object_detection_2d_data_generator import DataGenerator

input_format = ["image_name", "class_id", "xmin", "xmax", "ymin", "ymax"] # order of columns in the annotation csv

# Load validation data
validation_image_dir = "../FLIR_ADAS/validation/PreviewData"
validation_annotation_csv = "../validation.csv"

validation_dataset = DataGenerator()
validation_dataset.parse_csv(
    validation_image_dir,
    validation_annotation_csv,
    input_format
)
print("{} images in validation set.".format(validation_dataset.get_dataset_size()))

# Load training data
training_image_dir = "../FLIR_ADAS/training/PreviewData"
training_annotation_csv = "../training.csv"

training_dataset = DataGenerator()
training_dataset.parse_csv(
    training_image_dir,
    training_annotation_csv,
    input_format,
    include_classes=classes
)
print("{} images in training set.".format(training_dataset.get_dataset_size()))

1360 images in validation set.
5838 images in training set.


In [0]:
! ls *.h5

ssd300_alpha4_epoch-101_loss-8.0854_val_loss-6.7505.h5
ssd300_alpha4_epoch-102_loss-8.0846_val_loss-6.4888.h5
ssd300_alpha4_epoch-103_loss-8.0627_val_loss-6.4837.h5
ssd300_alpha4_epoch-105_loss-7.9965_val_loss-6.4393.h5
ssd300_alpha4_epoch-108_loss-7.8640_val_loss-6.4329.h5
ssd300_alpha4_epoch-111_loss-7.8159_val_loss-6.3716.h5
ssd300_alpha4_epoch-113_loss-7.9329_val_loss-6.3504.h5
ssd300_alpha4_epoch-114_loss-7.8479_val_loss-6.3400.h5
ssd300_alpha4_epoch-116_loss-7.7760_val_loss-6.3312.h5
ssd300_alpha4_epoch-121_loss-7.7476_val_loss-6.3080.h5
ssd300_alpha4_epoch-91_loss-7.7909_val_loss-6.3012.h5
ssd300_alpha4_epoch-92_loss-9.1434_val_loss-7.0649.h5
ssd300_alpha4_epoch-95_loss-8.7184_val_loss-7.0337.h5
ssd300_alpha4_epoch-97_loss-8.5773_val_loss-6.8645.h5
ssd300_geometric_aug_only_epoch-30_loss-4.4751_val_loss-3.8531.h5
ssd300_geometric_aug_only_finetuning_epoch-62_loss-3.7001_val_loss-3.1833.h5
ssd300_geometric_aug_only_finetuning_epoch-82_loss-3.6181_val_loss-3.1497.h5
ssd300_noaug_e

In [0]:
from keras import backend as K
from keras.models import load_model

from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_L2Normalization import L2Normalization
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_loss_function.keras_ssd_loss import SSDLoss

def load_ssd(h5_filename):
  #Clear previously loaded models
  K.clear_session()
  
  #Fill in the custom objects so that Keras can load the model
  ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
  
  custom_objects = {
      'AnchorBoxes': AnchorBoxes,
      'L2Normalization': L2Normalization,
      'DecodeDetections': DecodeDetections,
      'compute_loss': ssd_loss.compute_loss}
  
  #Load the model
  model = load_model(h5_filename, custom_objects)
  print("Model loaded:", h5_filename)
  
  return model
  
model = load_ssd("ssd300_alpha4_epoch-121_loss-7.7476_val_loss-6.3080.h5")

Model loaded: ssd300_alpha4_epoch-121_loss-7.7476_val_loss-6.3080.h5


In [0]:
from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder

img_height = 300
img_width = 300
n_classes = 80 # COCO classes

scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5],
                         [1.0, 2.0, 0.5]]

two_boxes_for_ar1=True
steps=[8, 16, 32, 64, 100, 300]
offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
clip_boxes=False
variances=[0.1, 0.1, 0.2, 0.2]
normalize_coords=True

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios_per_layer,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

In [0]:
from data_generator.object_detection_2d_geometric_ops import Resize, ResizeRandomInterp
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDExpand, SSDRandomCrop
from data_generator.object_detection_2d_geometric_ops import RandomFlip
from data_generator.object_detection_2d_image_boxes_validation_utils import BoxFilter

import cv2

mean_color = [128, 128, 128]

# For validation images
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# SSD augmentation chain without photometric ops
expand = SSDExpand(background=mean_color)
random_crop = SSDRandomCrop()
random_flip = RandomFlip(dim='horizontal', prob=0.5)
box_filter = BoxFilter(check_overlap=False, check_min_area=False, check_degenerate=True)
resize_random_interp = ResizeRandomInterp(
    height=img_height,
    width=img_width,
    interpolation_modes=[cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_AREA, cv2.INTER_LANCZOS4],
    box_filter=box_filter)

In [0]:
# Create training and validation generators
batch_size = 8

train_generator = training_dataset.generate(
    batch_size=batch_size,
    shuffle=True,
    transformations=[convert_to_3_channels, expand, random_crop, random_flip, resize_random_interp],
    label_encoder=ssd_input_encoder,
    returns={'processed_images', 'encoded_labels'},
    keep_images_without_gt=False)

val_generator = validation_dataset.generate(
    batch_size=batch_size,
    shuffle=False,
    transformations=[convert_to_3_channels, resize],
    label_encoder=ssd_input_encoder,
    returns={'processed_images', 'encoded_labels'},
    keep_images_without_gt=False)

In [0]:
# Define model callbacks.

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TerminateOnNaN

model_checkpoint = ModelCheckpoint(
    filepath='ssd300_alpha4_singlecat_person_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_delta=0.0001)

terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             reduce_lr_on_plateau,
             terminate_on_nan]

In [0]:
from keras.optimizers import Adam, SGD

# Set learning rate
lr = 0.001 / 2

adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = SGD()

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=4.0)

model.compile(optimizer=sgd, loss=ssd_loss.compute_loss)

In [0]:
# Train the model

from math import ceil

initial_epoch   = 121
final_epoch     = 300
steps_per_epoch = 1000

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=final_epoch,
    callbacks=callbacks,
    validation_data=val_generator,
    validation_steps=ceil(validation_dataset.get_dataset_size()/batch_size),
    initial_epoch=initial_epoch)

Epoch 122/300
1000/1000 [==============================] - 4259s 4s/step - loss: 12.8300 - val_loss: 19.9669

Epoch 00122: val_loss improved from inf to 19.96689, saving model to ssd300_alpha4_singlecat_person_epoch-122_loss-12.8299_val_loss-19.9669.h5
Epoch 123/300
1000/1000 [==============================] - 829s 829ms/step - loss: 11.5826 - val_loss: 20.2116

Epoch 00123: val_loss did not improve from 19.96689
Epoch 124/300
1000/1000 [==============================] - 830s 830ms/step - loss: 11.0812 - val_loss: 20.6250

Epoch 00124: val_loss did not improve from 19.96689
Epoch 125/300
1000/1000 [==============================] - 829s 829ms/step - loss: 11.0690 - val_loss: 19.2722

Epoch 00125: val_loss improved from 19.96689 to 19.27222, saving model to ssd300_alpha4_singlecat_person_epoch-125_loss-11.0691_val_loss-19.2722.h5
Epoch 126/300
1000/1000 [==============================] - 828s 828ms/step - loss: 11.0291 - val_loss: 18.7144

Epoch 00126: val_loss improved from 19.27222 to